# DynamoDB, RDB update query

In [1]:
# DB ops,dev 잘 구분해서 쿼리 날리기.
# 쿼리날리기전 테이블명 확인하고 쿼리.

import json, os
import pandas as pd
import boto3
from aico_lib.aico_db import DatabaseMgmt
from aico_lib.aico_constants import DB_TYPE, STAGE_TYPE, PRODUCT_CODE

os.environ['PRODUCT_CD'] = PRODUCT_CODE.aicando.name
os.environ['STAGE_TYPE'] = STAGE_TYPE.ops.name

# s3에서 평균풀이시간 가져오기
client = boto3.client("s3")
obj = client.get_object(Bucket="ops-aico-analysis-config", Key="common/meta_data/avg_solve_time/avg_solve_time_data.json")
S3_df_temp = pd.DataFrame(json.loads(obj["Body"].read()))
S3_df = S3_df_temp[['qst_id', 'target_time']]
# dict_data = S3_df.to_dict('records')

# AURORA RDB 조인데이터 가져오기
aurora_con = DatabaseMgmt(dbType=DB_TYPE.AICO_AURORA_POSTGRE)
select_sql = """
SELECT question_id, corner_code, subject_code, grade_code, semester_code FROM alps_db.question;
"""
corner_code_data = aurora_con.do_sql(reqSQL=select_sql)
corner_code_data_df = pd.DataFrame(corner_code_data['data'])


# S3, AURORA RDB JOIN
question_analysis_df = pd.merge(S3_df, corner_code_data_df, how ='left', left_on = 'qst_id', right_on = 'question_id')
question_analysis_df['target_time_min'] = ""
question_analysis_dict = question_analysis_df.to_dict('records')

In [ ]:
# AURORA RDB update
import json, os
from datetime import datetime
from pytz import timezone
import pandas as pd
import boto3
from aico_lib.aico_db import DatabaseMgmt
from aico_lib.aico_constants import DB_TYPE, STAGE_TYPE, PRODUCT_CODE

os.environ['PRODUCT_CD'] = PRODUCT_CODE.aicando.name
os.environ['STAGE_TYPE'] = STAGE_TYPE.ops.name
current_date = datetime.now(timezone("UTC")).strftime("%Y-%m-%d %H:%M:%S")

aurora_con = DatabaseMgmt(dbType=DB_TYPE.AICO_AURORA_POSTGRE)
insert_sql = """
INSERT INTO alps_db.question_analysis (product_code, question_id, target_time_analysis, target_time_min, corner_code, subject_code, grade_code, semester_code, update_datetime) VALUES(%(product_code)s, %(question_id)s, %(target_time_analysis)s, %(target_time_min)s, %(corner_code)s, %(subject_code)s, %(grade_code)s, %(semester_code)s, %(update_datetime)s) ON CONFLICT (product_code, question_id) DO UPDATE SET target_time_analysis=EXCLUDED.target_time_analysis, target_time_min=EXCLUDED.target_time_min, corner_code=EXCLUDED.corner_code, subject_code=EXCLUDED.subject_code, grade_code=EXCLUDED.grade_code, semester_code=EXCLUDED.semester_code, update_datetime=EXCLUDED.update_datetime;
"""
req_params=[]
for data in question_analysis_dict:
    req_param = {'product_code':'aicando', 'question_id':data['qst_id'], 'target_time_analysis':data['target_time'], 'target_time_min':1, 'corner_code':data['corner_code'], 'subject_code':data['subject_code'], 'grade_code':data['grade_code'], 'semester_code':data['semester_code'], 'update_datetime':current_date}
    req_params.append(req_param)
result = aurora_con.do_sql(reqSQL=insert_sql, reqParams=req_params)

In [ ]:
# DynamoDB update
# DB ops,dev 잘 구분해서 쿼리 날리기.
from datetime import datetime
from pytz import timezone
from source.question_meta import update_question_analysis_target_time_list

current_date = datetime.now(timezone("UTC")).strftime("%Y-%m-%d %H:%M:%S")

req_params=[]
for data in question_analysis_dict:

    req_param = {'productCode':'aicando', 'questionId':data['qst_id'], 'targetTimeAnalysis':str(data['target_time']), 'targetTimeMin':str(1), 'cornerCode':data['corner_code'], 'subjectCode':data['subject_code'], 'gradeCode':data['grade_code'], 'semesterCode':data['semester_code']}
    req_params.append(req_param)
update_question_analysis_target_time_list(product_code="aicando", data=req_params, stage="ops")

In [ ]:
# DynamoDB select
import json
import pandas as pd
import boto3

def __read_dynamo(table_nm, col_nm):
    """
    description:
        DynamoDB 에서 컬럼단위로 읽어오는 함수
    """
    table = boto3.resource("dynamodb").Table(table_nm)
    kwargs = {}
    if col_nm is not None:
        kwargs["ProjectionExpression"] = ", ".join(col_nm)
    response = table.scan(**kwargs)
    items = response["Items"]
    while "LastEvaluatedKey" in response:
        response = table.scan(ExclusiveStartKey=response["LastEvaluatedKey"], **kwargs)
        items.extend(response["Items"])
    return pd.DataFrame(items).drop_duplicates()

new_target_time = __read_dynamo("ops-service-aicando-mt-question-analysis",["question_id", "analysis_target_time", "subject_code", "corner_code", "grade_code", "semester_code"])
new_target_time
# new_corner_code = __read_dynamo("ops-service-aicando-mt-question",["question_id", "subject_code", "corner_code", "grade_code", "semester_code"])
# total_values = pd.merge(new_target_time, new_corner_code, how ='left', left_on = 'question_id', right_on = 'question_id')

In [ ]:
# 발전 : 03, 010_03, 020_03 / 메타에 존재 : 03
# 기본 : 02, 010_02, 020_02 / 메타에 존재 : 02